# Setup

In [1]:
using RigidBodyDynamics
using ForwardDiff
using Compat.Test

INFO: Recompiling stale cache file /home/twan/code/RigidBodyDynamics/lib/v0.6/RigidBodyDynamics.ji for module RigidBodyDynamics.


# Jacobians with respect to $q$ and $v$ - the simple way

First, we'll load our trusty double pendulum from a URDF:

In [2]:
const mechanism = parse_urdf(Float64, "../test/urdf/Acrobot.urdf")
remove_fixed_tree_joints!(mechanism)
mechanism

Spanning tree:
Vertex: world (root)
  Vertex: upper_link, Edge: shoulder
    Vertex: lower_link, Edge: elbow
No non-tree joints.

Of course, we can create a `MechanismState` for the double pendulum, and compute its momentum in some random state:

In [3]:
float64state = MechanismState(mechanism)
rand!(float64state)
const q = configuration(float64state) # for future use
const v = velocity(float64state) # for future use
momentum(float64state)

Momentum expressed in "world":
angular: [0.340081, 2.85725, 0.00728807], linear: [-0.049245, 0.0, 1.43793]

But now suppose we want the Jacobian of momentum with respect to the joint velocity vector $v$. We can do this using the `ForwardDiff.Dual` type and the `ForwardDiff.jacobian` function. The ForwardDiff package implements forward-mode [automatic differentiation](https://en.wikipedia.org/wiki/Automatic_differentiation).

To use `ForwardDiff.jacobian` we need to create a function that maps `v` (as a `Vector`) to momentum (as a `Vector`):

In [4]:
function mom(v::AbstractVector)
    # create a `MechanismState` that can handle the element type of `v` (which will be some `ForwardDiff.Dual`):
    T = eltype(v)
    state = MechanismState{T}(mechanism)
    
    # set the state variables:
    set_configuration!(state, q)
    set_velocity!(state, v)
    
    # return momentum converted to an `Array` (as this is the format that ForwardDiff expects)
    Array(momentum(state))
end

mom (generic function with 1 method)

Let's first check that the function returns the same thing we got from `float64state`:

In [5]:
@test mom(v) == Array(momentum(float64state))

Test Passed

That works, so now let's compute the Jacobian with `ForwardDiff`:

In [6]:
J = ForwardDiff.jacobian(mom, v)

6×2 Array{Float64,2}:
  0.562887    0.248262 
  4.52669     2.26184  
  0.0520363  -0.0294264
 -0.258276    0.117705 
  0.0         0.0      
  2.44516     0.993049 

At this point we note that the matrix `J` is simply the momentum matrix (in world frame) of the `Mechanism`. In this case, RigidBodyDynamics.jl has a specialized algorithm for computing this matrix, so let's verify the results:

In [7]:
A = momentum_matrix(float64state)

MomentumMatrix expressed in "world":
[0.562887 0.248262; 4.52669 2.26184; 0.0520363 -0.0294264; -0.258276 0.117705; 0.0 0.0; 2.44516 0.993049]

In [8]:
@test J ≈ Array(A) atol = 1e-12

Test Passed

Gradients with respect to $q$ can be computed in similar fashion.

# Improving performance

Ignoring the fact that we have a specialized method available, let's look at the performance of using `ForwardDiff.jacobian`.

In [9]:
using BenchmarkTools
@benchmark ForwardDiff.jacobian($mom, $v)

BenchmarkTools.Trial: 
  memory estimate:  45.81 KiB
  allocs estimate:  685
  --------------
  minimum time:     111.660 μs (0.00% GC)
  median time:      117.335 μs (0.00% GC)
  mean time:        127.553 μs (6.56% GC)
  maximum time:     4.528 ms (95.08% GC)
  --------------
  samples:          10000
  evals/sample:     1

That's not great. Note all the allocations. We can do better by making the following modifications:

1. use an in-place version of the `jacobian` function, `ForwardDiff.jacobian!`
2. reimplement our `mom` function to be in-place as well
3. don't create a new `MechanismState` every time

The third point is especially important; creating a `MechanismState` object is expensive!

To facillitate reuse of `MechanismState`s while keeping the code nice and generic, we can use a `StateCache` object.
`StateCache` is a container that stores `MechanismState`s of various types (associated with one `Mechanism`), and will ease the process of using `ForwardDiff`.
Creating one is easy:

In [10]:
const statecache = StateCache(mechanism)

StateCache{…}(…)

`MechanismState`s of a given type can be accessed as follows (note that if a `MechanismState` of a certain type is already available, it will be reused):

In [11]:
float32state = statecache[Float32]

MechanismState{Float32, Float64, Float64, …}(…)

In [12]:
@test float32state === statecache[Float32]

Test Passed

Now we'll use the `StateCache` to reimplement `mom`, making it in-place as well:

In [13]:
function mom!(out::AbstractVector, v::AbstractVector)
    # retrieve a `MechanismState` that can handle the element type of `v`:
    T = eltype(v)
    state = statecache[T]
    
    # set the state variables:
    set_configuration!(state, q)
    set_velocity!(state, v)
    
    # compute momentum and store it in `out`
    m = momentum(state)
    out[:] = [angular(m); linear(m)]
end

mom! (generic function with 1 method)

Check that the in-place version works as expected on `Float64` inputs:

In [14]:
const out = zeros(6) # where we'll be storing our results
mom!(out, v)
@test out == Array(momentum(float64state))

Test Passed

And use `ForwardDiff.jacobian!` to compute the Jacobian:

In [15]:
const result = DiffResults.JacobianResult(out, v)
const config = ForwardDiff.JacobianConfig(mom!, out, v)
ForwardDiff.jacobian!(result, mom!, out, v, config)
J = DiffResults.jacobian(result)
@test J ≈ Array(A) atol = 1e-12

Test Passed

Let's check the performance again:

In [16]:
@benchmark ForwardDiff.jacobian!($result, $mom!, $out, $v, $config)

BenchmarkTools.Trial: 
  memory estimate:  176 bytes
  allocs estimate:  2
  --------------
  minimum time:     2.385 μs (0.00% GC)
  median time:      2.421 μs (0.00% GC)
  mean time:        2.501 μs (0.00% GC)
  maximum time:     6.104 μs (0.00% GC)
  --------------
  samples:          10000
  evals/sample:     9

That's much better (the remaining allocations are due to the implementation of `StateCache`).